# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

Ans: 從訓練集的結果看來似乎標籤編碼比較理想。  
1.主要是均值編碼套用在分類問題的標籤進行均值轉換很類似計次的類別特徵轉換，帶來的問題就是邏輯回歸的訓練結果在交叉驗證之下都出現overfitting的現象。  
2.但在回歸的類別均值轉換就可以看到訓練集的擬合程度比標籤方式好時間也較快!!


In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_le = df.copy()
le = LabelEncoder()
for col in df_le.columns:
    df_le[col]=le.fit_transform(df_le[col])
LR = LogisticRegression()
train_X = df_le[:train_num]
start = time.time()
le_result = cross_val_score(LR, train_X, train_Y, cv=5)
time_duration = time.time() -start
print(f'score:{le_result.mean()} with time duration:{time_duration}')

score:0.780004837244799 with time duration:0.07895469665527344


C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be ch

In [4]:
# 均值編碼 + 邏輯斯迴歸
df_mean = df.copy()
df_mean = pd.concat([df_mean[:train_num], train_Y], axis=1)

for col in object_features:
    df_tmp = pd.DataFrame()
    df_tmp[[col, col+'_mean']] = df_mean.groupby([col])['Survived'].mean().reset_index()
    df_mean = pd.merge(df_mean, df_tmp, on=col, how='left')
    df_mean = df_mean.drop([col], axis=1)
df_mean = df_mean.drop('Survived', axis = 1)
print(df_mean[:5])
LR = LogisticRegression(C=0.00001)
start = time.time()
train_X_mean = df_mean[:train_num]
mean_label_result = cross_val_score(LR, train_X_mean, train_Y, cv=5)
time_duration = time.time() - start
print(f'score:{mean_label_result.mean()} with time duration:{time_duration} sec')

C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


   Name_mean  Sex_mean  Ticket_mean  Cabin_mean  Embarked_mean
0          0  0.188908          0.0    0.299854       0.336957
1          1  0.742038          1.0    1.000000       0.553571
2          1  0.742038          1.0    0.299854       0.336957
3          1  0.742038          0.5    0.500000       0.336957
4          0  0.188908          0.0    0.299854       0.336957
score:0.9988826815642458 with time duration:0.025983572006225586 sec


C:\Users\crea\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [5]:
df_mean.describe()

,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
count,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,0.383838,0.383838,0.383838,0.383838
std,0.486592,0.264391,0.451338,0.253065,0.088795
min,0.000000,0.188908,0.000000,0.000000,0.336957
25%,0.000000,0.188908,0.000000,0.299854,0.336957
50%,0.000000,0.188908,0.000000,0.299854,0.336957
75%,1.000000,0.742038,1.000000,0.299854,0.389610
max,1.000000,0.742038,1.000000,1.000000,1.000000
